## Enabling Product Quantization(PQ) Vector Compression for your Class

In order to compress vectors using PQ you need:

**1.** Connect to a Weaviate instance and create a Collection

**2.** Add datapoints to the class - it is recommended to add atleast 10k-100k objects to Weaviate before enabling PQ

**3.** Enable PQ by updating the Collection config (This will take the datapoints and vectors already added to Weaviate and will train the PQ algorithm on them - learning centroids that can be used to compress current and any future added vectors):
    
    a. You can specify the `trainingLimit` which will allow you to dictate how many of the added vectors will be used to train the centroids. By default this will take upto the first 100k objects added to Weaviate
    
    b. You can specify the `segments` to use which will specify how many pieces to quantize the vectors into. This will dictate the compression rate.

### 1. Connect to the Weaviate instance:

In [ ]:
import weaviate, os

openai_key = os.environ["OPENAI_API_KEY"] # Replace with your OpenAI key

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(
  headers={
    "X-OpenAI-Api-Key": openai_key
  }
)

# Option 2
# Connect to your Weaviate Client Service cluster
# client = weaviate.connect_to_wcs(
#     cluster_id="WCS-CLUSTER-ID", # Replace with your WCS cluster ID
#     auth_credentials=weaviate.AuthApiKey(
#       api_key="WCS-API-KEY" # Replace with your WCS API KEY
#     ),
#     headers={
#       "X-OpenAI-Api-Key": openai_key
#     }
# )

client.is_ready()

## Create a collection
By default PQ will be `disabled`, as it needs to be enabled after enough data is loaded.

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="question", data_type=wc.DataType.TEXT), 
        wc.Property(name="answer", data_type=wc.DataType.TEXT),
        wc.Property(name="round", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

### 2. Add data to the instance:

In [ ]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))

In [ ]:
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

items = []
for o in data:
    items.append(
        {
            'question':o["Question"],
            'answer':o["Answer"],
            'round':o["Round"]
        }
    )

    # insert items every time we have 100 
    if(len(items) == 100):
        response = jeopardy.data.insert_many(items)
        items.clear()

        # print errors if any
        if(response.has_errors):
            print(response.errors)

# insert remaining items
if(len(items)>0):
    response = jeopardy.data.insert_many(items)
    items.clear()

    # print errors if any
    if(response.has_errors):
        print(response.errors)

print("Data import complete")

In [ ]:
jeopardy.aggregate.over_all()

Perform a vector search:

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="spicy food recipes",
    return_metadata=["distance"],
    limit=2
)

for item in response.objects:
    print("Distance:", item.metadata.distance)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### 3. Enable PQ by updating the Collection config:

In [ ]:
import weaviate.classes.config as wc

jeopardy.config.update(
    vector_index_config=wc.Reconfigure.vector_index(
        pq_enabled=True,
        pq_training_limit=10000, # (optional) Number of vectors to be used to train PQ. Default value: 100k vectors
        pq_segments=96 #how many segments to break/quantize the vector representation into - has to be an integer multiple of vector dimension
    )
)

print("PQ enabled for JeopardyQuestion collection. Check your logs for progress.")

Your Weaviate instance will then enable compression and if you're monitoring the instance it will log the following:


```bash
product_quantization_compression-weaviate-1  | {"action":"compress","level":"info","msg":"switching to compressed vectors","time":"2023-11-13T21:10:52Z"}

product_quantization_compression-weaviate-1  | {"action":"compress","level":"info","msg":"vector compression complete","time":"2023-11-13T21:10:53Z"}
```

Re-run the same vector search now on PQ compressed vectors, (Rescoring is enabled by default)

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="spicy food recipes",
    return_metadata=["distance"],
    limit=2
)

for item in response.objects:
    print("Distance:", item.metadata.distance)
    print("Data:", json.dumps(item.properties, indent=2), "\n")